In [3]:
import pandas as pd
import folium
from pyproj import Transformer

In [4]:
stations = pd.read_csv('netatmo_stations.csv', delimiter=',')
stations.head()

,device_id,module_id,lon,lat
0,70:ee:50:7a:8d:66,02:00:00:7a:82:20,-0.748171,41.636124
1,70:ee:50:3f:63:68,02:00:00:3e:e0:04,-0.833446,41.671643
2,70:ee:50:3f:18:7e,02:00:00:3c:ec:c0,-0.774684,41.685630
3,70:ee:50:2b:58:c2,02:00:00:2a:db:82,-0.803851,41.712556
4,70:ee:50:36:cf:1c,02:00:00:37:03:a4,-0.797466,41.743163


In [7]:
zgz_coords = (41.653858, -0.876258)
map = folium.Map(location=zgz_coords, zoom_start=13)

# Iterate over each station
for index, station in stations.iterrows():
    lon = station['lon']
    lat = station['lat']

    # Add a marker for the point
    folium.Marker([lat, lon], popup=station['device_id']).add_to(map)

# If running in a Jupyter Notebook, display the map inline
map